## 📘 **Title**: `consolidation_all.ipynb`

### 🧾 **Description**

This notebook automates the process of loading **multiple CSV files across multiple consolidation snapshots** of acts and regulations into a Neo4j graph database. It performs the following:

* Connects to Neo4j using credentials stored in environment variables.
* Iterates through folders containing `rank_*.csv` files for each consolidation snapshot.
* Clears existing nodes with the same label (if any) before loading new ones.
* Loads each CSV file into Neo4j using `LOAD CSV` and Cypher.
* Creates both **vector indexes** and **text indexes** on the loaded nodes to support semantic and keyword search.

Each consolidation folder (e.g., `consol_42`) gets its own unique label like `consolidation_42_act` or `consolidation_42_reg`.

---

### 📂 **Before You Run This Notebook**

You need to copy all the CSV files into the Neo4j container so that they are accessible via the `file:///` protocol.

#### 🔧 Step-by-step Instructions:

1. **Locate your CSV folders**
   Typically under:
   `./Processed_embeddings/acts/consol_*/rank_*.csv`
   and
   `./Processed_embeddings/regs/consol_*/rank_*.csv`

2. **Copy the folders into the Neo4j container's import directory**
   Assuming your container is named `neo4j`, run the following (adjust path accordingly):

   ```bash
   docker cp ./Processed_embeddings neo4j:/var/lib/neo4j/import/
   ```

   Or if using **Podman**:

   ```bash
   podman cp ./Processed_embeddings neo4j:/var/lib/neo4j/import/
   ```

3. **Verify inside the container (optional)**:

   ```bash
   docker exec -it neo4j bash
   ls /var/lib/neo4j/import/Processed_embeddings/acts/
   ```

4. **Ensure APOC is enabled and the import path is correct**
   Make sure `apoc.import.file.enabled=true` and `dbms.directories.import=/var/lib/neo4j/import` are set in your `neo4j.conf`.

In [ ]:
import os
import glob
import re
from tqdm.notebook import tqdm  # For Jupyter notebooks specifically

In [ ]:
# Step 2: Load CSV
import pandas as pd

In [ ]:
NEO4J_URI = 'bolt://' + os.getenv('NEO4J_HOST') + ':7687'
NEO4J_USERNAME = os.getenv('NEO4J_USER')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = 'neo4j' #os.getenv('NEO4J_DB')
print(NEO4J_URI)
print(NEO4J_DATABASE)

In [ ]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

In [ ]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [ ]:
# Base directory for embeddings
EMBEDDINGS_DIR = "./Processed_embeddings"

In [ ]:
# Dictionary of consolidation dates
consolidation_dates = {
    # Acts
    "14": "2006-02-13", "15": "2006-07-11", "16": "2007-01-30", "17": "2007-09-05",
    "18": "2008-02-07", "19": "2008-12-12", "20": "2009-11-30", "21": "2010-10-27",
    "22": "2011-09-22", "23": "2012-08-03", "24": "2013-05-31", "25": "2013-12-31",
    "26": "2014-05-30", "27": "2014-12-10", "28": "2015-06-17", "29": "2016-01-06",
    "30": "2016-09-07", "31": "2017-04-05", "32": "2017-12-06", "33": "2018-07-11",
    "34": "2019-01-02", "35a": "2019-09-18", "35b": "2020-06-19", "36": "2020-12-02", 
    "37": "2021-07-14", "38": "2021-12-09", "39": "2022-06-08", "40": "2022-12-01", 
    "41": "2023-09-13", "42": "2024-03-11", "43": "2024-10-15",
    
    # Regs
    "68": "2005-12-09", "69": "2006-04-21", "70": "2006-08-25", "71": "2007-01-02",
    "72": "2007-05-04", "73": "2007-08-31", "74": "2008-01-04", "75": "2008-05-02",
    "76": "2008-09-05", "77": "2009-01-09", "78": "2009-05-15", "79": "2009-09-18",
    "80": "2010-01-15", "81": "2010-05-08", "82": "2010-09-11", "83": "2011-01-14",
    "84": "2011-05-20", "85": "2011-09-24", "86": "2012-01-20", "87": "2012-05-25",
    "88": "2012-09-25", "89": "2012-12-31", "90": "2013-04-30", "91": "2013-09-10",
    "92": "2013-12-31", "93": "2014-04-22", "94": "2014-08-26", "95": "2014-12-31",
    "96": "2015-05-05", "97": "2015-09-08", "98": "2015-12-31", "99": "2016-05-03",
    "100": "2016-09-06", "101": "2016-12-31", "102": "2017-04-25", "103": "2017-12-31",
    "104": "2018-12-31", "105": "2019-12-31", "106": "2020-12-31", "107": "2021-12-31",
    "108": "2022-12-31", "109": "2023-12-31", "110": "2024-12-31"
}

In [ ]:
# Function to clear existing nodes with a specific label
def clear_existing_nodes(consol_id, source_type):
    """Delete all existing nodes for a specific consolidation"""
    node_label = f"consolidation_{consol_id}_{source_type}"
    
    # First check if nodes exist
    check_query = f"""
    MATCH (n:{node_label})
    RETURN count(n) as count
    """
    
    result = kg.query(check_query)
    count = result[0]['count'] if result else 0
    
    if count == 0:
        print(f"No existing nodes found with label {node_label}. Nothing to delete.")
        return 0
    
    print(f"Found {count} existing nodes with label {node_label}. Deleting...")
    
    # Delete all nodes with this label
    delete_query = f"""
    MATCH (n:{node_label})
    DETACH DELETE n
    """
    
    kg.query(delete_query)
    print(f"✅ Successfully deleted {count} nodes with label {node_label}")
    return count


In [ ]:
# Function to process a single CSV file
def process_csv_file(file_path, consol_id, source_type):
    """Process a single CSV file and import it into Neo4j"""
    # Get consolidation date
    consol_date = consolidation_dates.get(str(consol_id), "2000-01-01")
    
    # Create a label for this consolidation
    node_label = f"consolidation_{consol_id}_{source_type}"
    
    # Create Cypher query
    cypher = f"""
    LOAD CSV WITH HEADERS FROM 'file:///{file_path}' AS row
    CALL (row) {{
      WITH row
      CREATE (s:{node_label} {{
      }})
      SET
          s.title = row.title,
          s.content = row.content,
          s.act_title = row.act_id,
          s.reg_title = row.reg_title,
          s.url = row.url,
          s.content = row.chunk_text,
          s.section_number = row.section_number,
          s.section_title = row.section_title,
          s.source_rank = toInteger(COALESCE(row.source_rank, "0")),
          s.timestamp = row.timestamp,
          s.snapshot_id = "{consol_id}",
          s.snapshot_date = date("{consol_date}"),
          s.source_type = "{source_type}",
          s.tokens = row.tokens,
          s.token_chunks = row.token_chunk,
          s.embedding = CASE 
              WHEN row.embedding STARTS WITH '[' AND row.embedding ENDS WITH ']'
              THEN apoc.convert.fromJsonList(row.embedding)
              ELSE null
          END
    }} IN 1 CONCURRENT TRANSACTIONS OF 500 ROWS
    """
    
    return kg.query(cypher)

In [ ]:
# Function to process a consolidation folder
def process_consolidation_folder(folder_path):
    """Process all CSV files in a consolidation folder"""
    # Extract folder name and type
    folder_name = os.path.basename(folder_path)
    parent_dir = os.path.basename(os.path.dirname(folder_path))
    
    # Determine if this is acts or regs
    source_type = "act" if parent_dir == "acts" else "reg"
    
    # Extract consolidation ID
    match = re.search(r'consol_(\d+\w*)', folder_name)
    if not match:
        print(f"Skipping folder with invalid name: {folder_name}")
        return
        
    consol_id = match.group(1)
    
    print(f"Processing {folder_name} ({source_type})")

    # Clear all existing nodes for this consolidation first
    clear_existing_nodes(consol_id, source_type)
    
    # Get all CSV files
    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    print(f"Found {len(csv_files)} CSV files")
    
    # Process each file
    for csv_file in tqdm(csv_files, desc=f"Processing {folder_name}"):
        file_name = os.path.basename(csv_file)
        rel_path = EMBEDDINGS_DIR + '/' + os.path.join(parent_dir, folder_name, file_name)
        
        print(f"Importing {file_name} from {rel_path}...")
        process_csv_file(rel_path, consol_id, source_type)
        print(f"✅ Completed import for {file_name}")
    
    # Create vector index for this consolidation
    create_vector_index(f"consolidation_{consol_id}_{source_type}")
    
    print(f"Completed processing {folder_name}")

In [ ]:
# Function to create a vector index
def create_vector_index(label):
    """Create a vector index for a specific label"""
    # Check if there are nodes with this label
    check_query = f"""
    MATCH (n:{label})
    RETURN count(n) as count
    """
    
    result = kg.query(check_query)
    count = result[0]['count'] if result else 0
    
    if count == 0:
        print(f"No nodes found with label {label}. Skipping index creation.")
        return
    
    print(f"Creating vector index for {label} ({count} nodes)...")
    
    # Create vector index
    vector_index_query = f"""
    CREATE VECTOR INDEX {label}_embedding_idx IF NOT EXISTS
    FOR (n:{label})
    ON n.embedding
    OPTIONS {{
      indexConfig: {{
        `vector.dimensions`: 384,
        `vector.similarity_function`: "cosine"
      }}
    }}
    """
    
    kg.query(vector_index_query)
    print(f"✅ Created vector index for {label}")
    
    # Create regular indexes for text properties
    kg.query(f"""
    CREATE INDEX {label}_title_idx IF NOT EXISTS
    FOR (n:{label})
    ON (n.title)
    """)
    print(f"✅ Created title index for {label}")

In [ ]:
def process_all_in_folder(folder_type="acts", start_id=None, end_id=None):
    """Process all consolidation folders of a specific type"""
    base_path = os.path.join('../' + EMBEDDINGS_DIR, folder_type)
    print(f"Processing {folder_type} folder: {base_path}")
    
    # Find all consolidation folders
    consol_folders = glob.glob(os.path.join(base_path, "consol_*"))
    print(f"Found {len(consol_folders)} consolidation folders")
    
    # Filter by ID range if specified
    if start_id is not None or end_id is not None:
        filtered_folders = []
        for folder in consol_folders:
            folder_name = os.path.basename(folder)
            match = re.search(r'consol_(\d+\w*)', folder_name)
            if not match:
                continue
                
            consol_id = match.group(1)
            
            # Handle numeric IDs
            try:
                numeric_id = int(consol_id)
                if (start_id is None or numeric_id >= start_id) and (end_id is None or numeric_id <= end_id):
                    filtered_folders.append(folder)
            except ValueError:
                # Handle non-numeric IDs like 35a, 35b
                filtered_folders.append(folder)
        
        consol_folders = filtered_folders
        print(f"Filtered to {len(consol_folders)} folders in ID range")
    
    # Process each folder
    for folder in consol_folders:
        process_consolidation_folder(folder)
    
    print(f"Completed processing all {folder_type} folders")

In [ ]:
# Example usages:

# Process a single consolidation folder
# process_consolidation_folder("./Processed_embeddings/acts/consol_42")

# Process all acts consolidations
# process_all_in_folder("acts")

# Process a specific range of acts consolidations
# process_all_in_folder("acts", start_id=40, end_id=43)

# Process all regulations consolidations
# process_all_in_folder("regs")

In [ ]:
process_consolidation_folder("..//Processed_embeddings/acts/consol_42")

In [ ]:
process_all_in_folder("acts")

In [ ]:
process_all_in_folder("regs")

In [134]:
# Example vector search function
def vector_search(query_embedding, label, limit=5):
    """
    Perform a vector search against a specific label
    
    Args:
        query_embedding: The embedding vector for the query
        label: The node label to search (e.g., "consolidation_42_act")
        limit: Maximum number of results to return
    
    Returns:
        List of matching nodes
    """
    search_query = f"""
    MATCH (n:{label})
    WHERE n.embedding IS NOT NULL
    WITH n, gds.similarity.cosine(n.embedding, $query_embedding) AS score

    ORDER BY score DESC
    LIMIT {limit}
    RETURN n.act_title AS act_title, n.reg_title as reg_title, n.section_title as section_title, n.content AS content, score
    """
    return kg.query(search_query, {"query_embedding": query_embedding})

# Example text search function
def text_search(text, label, limit=5):
    """
    Perform a text search against a specific label
    
    Args:
        text: The text to search for
        label: The node label to search (e.g., "consolidation_42_act")
        limit: Maximum number of results to return
    
    Returns:
        List of matching nodes
    """
    search_query = f"""
    MATCH (n:{label})
    WHERE n.content CONTAINS $text
    RETURN n.act_title AS title, n.content AS content
    LIMIT {limit}
    """
    
    return kg.query(search_query, {"text": text})

In [149]:
def vector_search_v2(query_embedding, label, limit=5):
    """
    Perform a vector search against a specific label using db.index.vector.queryNodes
    
    Args:
        query_embedding: The embedding vector for the query
        label: The node label to search (e.g., "consolidation_42_act")
        limit: Maximum number of results to return
    
    Returns:
        List of matching nodes
    """
    # Construct the index name based on the label
    index_name = f"{label}_embedding_idx"
    
    search_query = f"""
    CALL db.index.vector.queryNodes($index_name, $top_k, $query_embedding) 
    YIELD node, score
    RETURN 
        node.act_title AS act_title, 
        node.reg_title as reg_title, 
        node.title as section_title, 
        node.content AS content, 
        score
    ORDER BY score DESC
    """
    
    return kg.query(search_query, {
        "index_name": index_name,
        "top_k": limit,
        "query_embedding": query_embedding
    })

In [ ]:
text_search("offence act", 'consolidation_42_act', limit=5)

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [155]:
vector = embeddings.embed_query("1 despite section 5  ( 1 )  and any regulation made under section 4  ( 1 )   ( b )  ,  the minister may by order permit a board to do any of the following outside the region designated for the board :  a provide a health service ;  b locate a facility ;  c provide for a program of health service delivery .  2 if the minister makes an order under subsection  ( 1 )  ,  a the board is subject to any limits or conditions specified in the order ,  and b section 5 applies to the health service ,  facility or program as if the health service ,  facility or program were located in the region designated for the board .")

In [156]:
vector_search_v2(vector, 'consolidation_109_reg', limit=10)

[{'act_title': 'Health Professions Act',
  'reg_title': 'Health Professions General Regulation',
  'section_title': None,
  'content': '1 if the conditions of section 12 are met ,  the provincial health officer may make an order to do one or more of the following :  a authorize ,  but not require ,  a registrant to perform a specified activity or provide a specified service that the registrant would not otherwise be permitted to perform or provide under a health profession regulation ;  b modify or waive a requirement ,  standard ,  limit or condition set under a health profession regulation and authorize ,  but not require ,  a registrant to perform a specified activity or provide a specified service in accordance with the modification or waiver ;  c subject to subsection  ( 2 )  ,  authorize ,  but not require ,  a health professional who is not a registrant to perform a specified activity or to provide a specified service that ,  under a health profession regulation ,  may be perfor

In [ ]:
For this Act, if an offence is committed on the boundary of 2 or more territorial divisions or within 450 m of a boundary, or the offence was commenced in one territorial division and completed in another, the offence is deemed to be committed in any of the territorial divisions.